In [1]:
import numpy as np
import pandas as pd
import glob
from astropy.table import Table
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score
from sklearn.model_selection import ParameterGrid
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import hdbscan
import matplotlib.pyplot as plt
from itertools import combinations
import umap
from joblib import Parallel, delayed
import seaborn as sns; sns.set()
sns.set_theme(style="ticks")

/home/luisangel/luis-env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = Table.read("../Lomeli-data/catalog_all_bands_all_fovs_all_sources_106_rband.dat", format='ascii')

In [3]:
data

NUMBER,ALPHA,DELTA,u,uerr,g,gerr,r,rerr,i,ierr,z,zerr,F378,F378err,F395,F395err,F410,F410err,F430,F430err,F515,F515err,F660,F660err,F861,F861err,flags_i,fwhm_i,fwhm_psf_i,ellog_i,ellip_i,class_i,spread_i,flux_radius_i,area_i,mumax_i,mag_aper2_i,magerr_apper2_i,mag_aper4_i,magerr_apper4_i,mag_aper6_i,magerr_apper6_i,flux_aper6_u,fluxerr_aper6_u,flux_aper6_g,fluxerr_aper6_g,flux_aper6_r,fluxerr_aper6_r,flux_aper6_i,fluxerr_aper6_i,flux_aper6_z,fluxerr_aper6_z,flux_aper6_F378,fluxerr_aper6_F378,flux_aper6_F395,fluxerr_aper6_F395,flux_aper6_F410,fluxerr_aper6_F410,flux_aper6_F430,fluxerr_aper6_F430,flux_aper6_F515,fluxerr_aper6_F515,flux_aper6_F660,fluxerr_aper6_F660,flux_aper6_F861,fluxerr_aper6_F861,x,y,field,a_image,b_image,theta_image,kron_radius_i,flags_r,fwhm_r,fwhm_psf_r,ellog_r,ellip_r,class_r,spread_r,flux_radius_r,area_r,mumax_r,kron_radius_r
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64
5,41.707425223865236,-31.60953951032458,19.453903,0.10221078,17.678207,0.008456315,17.070164,0.005748007,16.682482,0.0050361827,16.36624,0.005675434,19.489212,0.10844446,20.44644,0.23599634,17.814161,0.053224858,16.828392,0.0261401,17.97462,0.023195434,17.164143,0.0071800356,16.585669,0.008681672,0,97.10646,1.35,1.5803512,0.36722928,0.028627958,0.022320295,9.773784,1964,18.066051,17.74692,0.008980873,16.27479,0.004586961,15.397849,0.0030643854,1.9030478,0.14541659,287.35037,1.5994353,596.5138,1.9053185,746.2087,2.1055903,553.5689,1.96315,1.3107454,0.12842599,2.0550382,0.22164163,10.284723,0.4932552,14.899448,0.5281288,37.278816,0.57456315,55.363686,0.22420436,181.2198,0.8964042,9614.776,857.39124,s24s28,20.334076,12.866808,-8.387699,3.5,0,28.233664,1.35,1.5803512,0.36722928,0.028625848,0.024721019,10.207849,1979,18.582102,3.5
6,42.169685789535045,-31.536274435360895,20.278736,0.06685978,18.145863,0.010459112,17.325224,0.0063154073,16.62676,0.004762817,16.10438,0.004709606,19.835825,0.076692484,19.71853,0.114814244,19.266186,0.06786906,18.604372,0.04120995,17.801802,0.019421658,17.076185,0.006600071,16.291069,0.0071237087,2,28.470156,1.35,5.8545294,0.82919204,0.028604275,0.014136691,14.382016,2918,18.448816,16.671814,0.004736057,15.530975,0.00290947,14.976801,0.0023534268,4.3138967,0.15126398,336.6122,1.6736062,784.2825,2.0954762,1099.7137,2.3831453,929.7292,2.2233984,3.5485754,0.13472958,4.0232134,0.22773635,14.988346,0.5063424,25.128761,0.5552191,62.9335,0.63140273,93.29202,0.26732874,295.4028,1.0174146,7038.8257,1348.9629,s24s28,27.765617,4.7425876,-64.89278,3.5,2,35.86831,1.35,5.8545294,0.82919204,0.028548878,0.016806884,15.45667,2995,19.180944,3.5
9,41.83725051548954,-31.610296215789248,16.72242,0.006870392,14.648617,0.001814241,13.828032,0.0011784958,13.586303,0.0010893539,13.472961,0.0012527949,16.554878,0.009415278,16.26182,0.01134155,15.432614,0.006411544,15.245046,0.005534445,14.60889,0.0035425338,13.802773,0.0013788448,13.481914,0.001767922,0,3.1743405,1.35,1.2570198,0.2044676,0.99676615,0.0010048964,1.6921986,511,15.025552,13.817455,0.0011493948,13.638036,0.0010801667,13.600438,0.0010966003,25.148527,0.1943041,1966.9094,3.287272,3814.383,4.08088,3906.8308,3.944965,2522.4358,3.090472,14.265941,0.16140062,19.616104,0.27075833,105.05555,0.7097966,141.65158,0.8004018,310.1512,1.0284169,397.0706,0.491051,863.3007,1.47838,8890.94,856.8943,s24s28,2.7917912,2.2209604,-4.292692,3.5,0,3.1876063,1.35,1.2570198,0.2044676,0.999676

In [4]:
combined_df = data.to_pandas()

In [5]:
for columns in combined_df.columns:
    print(columns)

NUMBER
ALPHA
DELTA
u
uerr
g
gerr
r
rerr
i
ierr
z
zerr
F378
F378err
F395
F395err
F410
F410err
F430
F430err
F515
F515err
F660
F660err
F861
F861err
flags_i
fwhm_i
fwhm_psf_i
ellog_i
ellip_i
class_i
spread_i
flux_radius_i
area_i
mumax_i
mag_aper2_i
magerr_apper2_i
mag_aper4_i
magerr_apper4_i
mag_aper6_i
magerr_apper6_i
flux_aper6_u
fluxerr_aper6_u
flux_aper6_g
fluxerr_aper6_g
flux_aper6_r
fluxerr_aper6_r
flux_aper6_i
fluxerr_aper6_i
flux_aper6_z
fluxerr_aper6_z
flux_aper6_F378
fluxerr_aper6_F378
flux_aper6_F395
fluxerr_aper6_F395
flux_aper6_F410
fluxerr_aper6_F410
flux_aper6_F430
fluxerr_aper6_F430
flux_aper6_F515
fluxerr_aper6_F515
flux_aper6_F660
fluxerr_aper6_F660
flux_aper6_F861
fluxerr_aper6_F861
x
y
field
a_image
b_image
theta_image
kron_radius_i
flags_r
fwhm_r
fwhm_psf_r
ellog_r
ellip_r
class_r
spread_r
flux_radius_r
area_r
mumax_r
kron_radius_r


In [6]:
m_x =  (combined_df["r"] >= 13) & (combined_df["r"] <= 22.5)
m_err = (combined_df["rerr"] <= 0.3) & (combined_df["gerr"] <= 0.3) & \
        (combined_df["ierr"] <= 0.3) & \
         (combined_df["F430err"] <= 0.3) & \
        (combined_df["F515err"] <= 0.3) & (combined_df["F660err"] <= 0.3) & \
        (combined_df["F861err"] <= 0.3) & (combined_df["zerr"] <= 0.3)

flags = (combined_df["flags_r"] == 0) & (combined_df["flags_i"] == 0)

mask = m_x & m_err & flags

df_cleanErr = combined_df[mask]
len(df_cleanErr)

327077

In [7]:
#Selecting columns
columns = ["r",
"g",
"i",
"z",
"F430",
"F515",
"F660",
"F861"]


In [8]:
df_mag = df_cleanErr[columns]
df_mag

,r,g,i,z,F430,F515,F660,F861
0,17.070164,17.678207,16.682482,16.366240,16.828392,17.974620,17.164143,16.585669
2,13.828032,14.648617,13.586303,13.472961,15.245046,14.608890,13.802773,13.481914
3,17.111893,17.397250,17.074059,16.979103,16.385063,17.409624,17.408487,17.508982
4,19.361853,19.980090,19.016304,18.795696,20.733458,19.546267,19.256630,18.891642
5,17.221579,18.326470,16.830046,16.648365,19.242409,18.357973,17.074144,16.697836
...,...,...,...,...,...,...,...,...
1749373,18.763690,19.661716,18.455479,18.277122,20.111898,19.648975,18.638176,18.348700
1749374,18.142593,18.686644,17.960854,17.891920,19.070671,18.506151,18.122720,17.877625
1749375,18.275616,19.581072,17.622740,17.352358,20.124254,19.617146,18.030436,17.426100
1749382,14.789453,15.418627,14.624699,14.555744,15.922544,15.268383,14.743403,14.579458


In [9]:
# Generate all combinations of magnitude columns
color_index_pairs = list(combinations(df_mag, 2))
len(color_index_pairs)

28

In [10]:
def calculate_earnings(df, index_pairs):
    for index_pair in index_pairs:
        color_index_name = f"{index_pair[0]} - {index_pair[1]}"
        df.loc[:, color_index_name] = df[index_pair[0]] - df[index_pair[1]]
    return df


In [11]:
df_colors_mag = calculate_earnings(df_mag, color_index_pairs)

/tmp/ipykernel_3834/2480231799.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, color_index_name] = df[index_pair[0]] - df[index_pair[1]]
/tmp/ipykernel_3834/2480231799.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, color_index_name] = df[index_pair[0]] - df[index_pair[1]]
/tmp/ipykernel_3834/2480231799.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [12]:
df_colors_mag

,r,g,i,z,F430,F515,F660,F861,r - g,r - i,...,z - F430,z - F515,z - F660,z - F861,F430 - F515,F430 - F660,F430 - F861,F515 - F660,F515 - F861,F660 - F861
0,17.070164,17.678207,16.682482,16.366240,16.828392,17.974620,17.164143,16.585669,-0.608043,0.387682,...,-0.462152,-1.608380,-0.797903,-0.219429,-1.146228,-0.335751,0.242723,0.810477,1.388951,0.578474
2,13.828032,14.648617,13.586303,13.472961,15.245046,14.608890,13.802773,13.481914,-0.820585,0.241729,...,-1.772085,-1.135929,-0.329812,-0.008953,0.636156,1.442273,1.763132,0.806117,1.126976,0.320859
3,17.111893,17.397250,17.074059,16.979103,16.385063,17.409624,17.408487,17.508982,-0.285357,0.037834,...,0.594040,-0.430521,-0.429384,-0.529879,-1.024561,-1.023424,-1.123919,0.001137,-0.099358,-0.100495
4,19.361853,19.980090,19.016304,18.795696,20.733458,19.546267,19.256630,18.891642,-0.618237,0.345549,...,-1.937762,-0.750571,-0.460934,-0.095946,1.187191,1.476828,1.841816,0.289637,0.654625,0.364988
5,17.221579,18.326470,16.830046,16.648365,19.242409,18.357973,17.074144,16.697836,-1.104891,0.391533,...,-2.594044,-1.709608,-0.425779,-0.049471,0.884436,2.168265,2.544573,1.283829,1.660137,0.376308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1749373,18.763690,19.661716,18.455479,18.277122,20.111898,19.648975,18.638176,18.348700,-0.898026,0.308211,...,-1.834776,-1.371853,-0.361054,-0.071578,0.462923,1.473722,1.763198,1.010799,1.300275,0.289476
1749374,18.142593,18.686644,17.960854,17.891920,19.070671,18.506151,18.122720,17.877625,-0.544051,0.181739,...,-1.178751,-0.614231,-0.230800,0.014295,0.564520,0.947951,1.193046,0.383431,0.628526,0.245095
1749375,18.275616,19.581072,17.622740,17.352358,20.124254,19.617146,18.030436,17.426100,-1.305456,0.652876,...,-2.771896,-2.264788,-0.678078,-0.073742,0.507108,2.093818,2.698154,1.586710,2.191046,0.604336
1749382,14.789453,15.418627,14.624699,14.555744,15.922544,15.268383,14.743403,14.579458,-0.629174,0.164754,...,-1.366800,-0.712639,-0.187658,-0.023714,0.654161,1.179142,1.343086,0.524980,0.688925,0.163944


In [13]:
# Drop magniytudes
df_colors = df_colors_mag.drop(columns=columns)

In [14]:
df_colors

,r - g,r - i,r - z,r - F430,r - F515,r - F660,r - F861,g - i,g - z,g - F430,...,z - F430,z - F515,z - F660,z - F861,F430 - F515,F430 - F660,F430 - F861,F515 - F660,F515 - F861,F660 - F861
0,-0.608043,0.387682,0.703924,0.241772,-0.904456,-0.093979,0.484495,0.995725,1.311967,0.849815,...,-0.462152,-1.608380,-0.797903,-0.219429,-1.146228,-0.335751,0.242723,0.810477,1.388951,0.578474
2,-0.820585,0.241729,0.355071,-1.417014,-0.780858,0.025259,0.346118,1.062314,1.175656,-0.596429,...,-1.772085,-1.135929,-0.329812,-0.008953,0.636156,1.442273,1.763132,0.806117,1.126976,0.320859
3,-0.285357,0.037834,0.132790,0.726830,-0.297731,-0.296594,-0.397089,0.323191,0.418147,1.012187,...,0.594040,-0.430521,-0.429384,-0.529879,-1.024561,-1.023424,-1.123919,0.001137,-0.099358,-0.100495
4,-0.618237,0.345549,0.566157,-1.371605,-0.184414,0.105223,0.470211,0.963786,1.184394,-0.753368,...,-1.937762,-0.750571,-0.460934,-0.095946,1.187191,1.476828,1.841816,0.289637,0.654625,0.364988
5,-1.104891,0.391533,0.573214,-2.020830,-1.136394,0.147435,0.523743,1.496424,1.678105,-0.915939,...,-2.594044,-1.709608,-0.425779,-0.049471,0.884436,2.168265,2.544573,1.283829,1.660137,0.376308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1749373,-0.898026,0.308211,0.486568,-1.348208,-0.885285,0.125514,0.414990,1.206237,1.384594,-0.450182,...,-1.834776,-1.371853,-0.361054,-0.071578,0.462923,1.473722,1.763198,1.010799,1.300275,0.289476
1749374,-0.544051,0.181739,0.250673,-0.928078,-0.363558,0.019873,0.264968,0.725790,0.794724,-0.384027,...,-1.178751,-0.614231,-0.230800,0.014295,0.564520,0.947951,1.193046,0.383431,0.628526,0.245095
1749375,-1.305456,0.652876,0.923258,-1.848638,-1.341530,0.245180,0.849516,1.958332,2.228714,-0.543182,...,-2.771896,-2.264788,-0.678078,-0.073742,0.507108,2.093818,2.698154,1.586710,2.191046,0.604336
1749382,-0.629174,0.164754,0.233709,-1.133091,-0.478930,0.046050,0.209995,0.793928,0.862883,-0.503917,...,-1.366800,-0.712639,-0.187658,-0.023714,0.654161,1.179142,1.343086,0.524980,0.688925,0.163944


# Umap

In [15]:
# Standarized the data
X_stand = StandardScaler().fit_transform(df_colors)

In [16]:
# Set up logging
import logging
logging.basicConfig(level=logging.INFO)


Try cross-validation

To perform cross-validation for choosing the optimal number of components in UMAP.


In [17]:
# Function to evaluate a single parameter combination
def evaluate_params(num_components, n_neighbors, X_train):
    logging.info(f"Evaluating: num_components={num_components}, n_neighbors={n_neighbors}")
    try:
        # Fit UMAP model
        reducer_ = umap.UMAP(n_neighbors=n_neighbors, n_components=num_components, random_state=42)
        X_train_trans = reducer_.fit_transform(X_train)

        # Cluster the transformed data using KMeans
        kmeans = KMeans(n_clusters=num_components, random_state=42)
        labels = kmeans.fit_predict(X_train_trans)

        # Evaluate performance on validation set using Silhouette Score
        silhouette = silhouette_score(X_train_trans, labels, random_state=42)
        return num_components, n_neighbors, silhouette, labels
    except Exception as e:
        logging.error(f"Error for num_components={num_components}, n_neighbors={n_neighbors}: {e}")
        return num_components, n_neighbors, -1, None

In [18]:
# Split data into training and validation sets
X_train, X_val = train_test_split(X_stand, test_size=0.2, random_state=42)

In [19]:
# Define the parameter ranges
num_components_range = [2, 3, 4, 5, 10, 20, 50]
n_neighbors_range = [5, 10, 20, 50, 100]

# Initialize variables to store the best results
best_silhouette_score = -1
best_num_components = None
best_n_neighbors = None
best_labels = None

In [20]:
# Batch processing to avoid memory issues
batch_size = 5
num_combinations = len(num_components_range) * len(n_neighbors_range)
batches = np.array_split(range(num_combinations), num_combinations // batch_size + 1)

In [21]:
for batch in batches:
    # Perform parallel computation for the current batch
    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(evaluate_params)(num_components_range[i // len(n_neighbors_range)],
                                 n_neighbors_range[i % len(n_neighbors_range)], X_train)
        for i in batch
    )

    # Process the results
    for num_components, n_neighbors, silhouette, labels in results:
        print(f"Number of components: {num_components}, Number of neighbors: {n_neighbors}, Silhouette Score: {silhouette}")
        if silhouette > best_silhouette_score:
            best_silhouette_score = silhouette
            best_num_components = num_components
            best_n_neighbors = n_neighbors
            best_labels = labels

print(f"Best Silhouette Score: {best_silhouette_score}")
print(f"Best number of components: {best_num_components}")
print(f"Best number of neighbors: {best_n_neighbors}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 55.6min remaining: 83.4min
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed: 56.3min remaining: 37.5min


Number of components: 2, Number of neighbors: 5, Silhouette Score: 0.41301482915878296
Number of components: 2, Number of neighbors: 10, Silhouette Score: 0.4428045153617859
Number of components: 2, Number of neighbors: 20, Silhouette Score: 0.4634738564491272
Number of components: 2, Number of neighbors: 50, Silhouette Score: 0.48282110691070557
Number of components: 2, Number of neighbors: 100, Silhouette Score: 0.4932107627391815


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 62.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 62.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 50.1min remaining: 75.2min
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed: 54.9min remaining: 36.6min


Number of components: 3, Number of neighbors: 5, Silhouette Score: 0.4206136167049408
Number of components: 3, Number of neighbors: 10, Silhouette Score: 0.45734673738479614
Number of components: 3, Number of neighbors: 20, Silhouette Score: 0.46811023354530334
Number of components: 3, Number of neighbors: 50, Silhouette Score: 0.4773942232131958
Number of components: 3, Number of neighbors: 100, Silhouette Score: 0.4834122657775879


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 56.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 56.8min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 46.3min remaining: 69.5min
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed: 51.0min remaining: 34.0min


...

[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 46.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 46.8min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 41.8min
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed: 45.8min remaining: 45.8min


Number of components: 20, Number of neighbors: 20, Silhouette Score: 0.33820775151252747
Number of components: 20, Number of neighbors: 50, Silhouette Score: 0.3403800427913666
Number of components: 20, Number of neighbors: 100, Silhouette Score: 0.34181299805641174
Number of components: 50, Number of neighbors: 5, Silhouette Score: 0.2989826500415802


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 50.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 50.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 40.4min
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed: 48.5min remaining: 48.5min


Number of components: 50, Number of neighbors: 10, Silhouette Score: 0.3038170039653778
Number of components: 50, Number of neighbors: 20, Silhouette Score: 0.295764297246933
Number of components: 50, Number of neighbors: 50, Silhouette Score: 0.3017258942127228
Number of components: 50, Number of neighbors: 100, Silhouette Score: 0.3032468855381012
Best Silhouette Score: 0.4932107627391815
Best number of components: 2
Best number of neighbors: 100


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 56.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 56.6min finished
